In [1]:
import bcrypt
import os

In [2]:
# Define the User data file
USER_DATA_FILE = "users.txt"

def hash_password(plain_text_password):
    # encode the password to bytes(bcrypt requires byte strings)
    password_byte = plain_text_password.encode('utf-8')
    # generate a salt using bcrypt.gensalt()
    salt = bcrypt.gensalt()
    # hash the password with the salt using bcrypt.hashpw()
    hashed_password = bcrypt.hashpw(password_byte, salt)
    # Decode the hash back to a string to store in a textfile.
    return hashed_password.decode('utf-8')


In [3]:
#implementing the Password verification function
def verify_password(plain_text_password, stored_hashed_password):
    """
    Verifies a plaintext password against a stored bcrypt hash.

    Args:
        plain_text_password (str): The password to verify.
        stored_hashed_password (str): The stored hash to check against.

    Returns:
        bool: True if the password matches the hash, False otherwise.
    """
    # Encode both the plaintext password and the stored hashed password to bytes
    password_bytes = plain_text_password.encode('utf-8')
    hashed_bytes = stored_hashed_password.encode('utf-8')
    # Use bcrypt.checkpw() to verify the password
    # This function extracts the salt from the hash and compares
    return bcrypt.checkpw(password_bytes, hashed_bytes)

In [4]:
# Test your hashing functions 
if __name__ == "__main__":
    #Temporary test code - remove after testing
    test_password = "SecurePassword123"
    hashed = hash_password(test_password)
    # Test hashing
    print(f"Original password: {test_password}")
    print(f"Hashed password: {hashed}")
    print(f"hash length: {len(hashed)}")
    # Test verification with corrct password 
    is_valid = verify_password(test_password, hashed)
    print(f"\nVerification with correct password: {is_valid}")
    # Test verification with incorrect password 
    is_valid = verify_password("WrongPassword", hashed)
    print(f"Verification with incorrect password: {is_valid}")

Original password: SecurePassword123
Hashed password: $2b$12$P89NCg/FvmkaNo5sJkJbNuzsabqHyEjq00.ZDjiEJhQvlWuXikEOu
hash length: 60

Verification with correct password: True
Verification with incorrect password: False


In [5]:
#Implement the registration function 
def register_user(username, password):
    """
    Registers a new user by hashing their password and storing credentials.

    Args:
        username (str): The username of the new account.
        plain_text_password (str): The plaintext password to hash and store.

    Returns:
        bool: True if registration successful, False if username already exists.
    """
    # Check if the username already exists
    if os.path.exists(USER_DATA_FILE):
        with open(USER_DATA_FILE, 'r') as file:
            for line in file:
                stored_username, _ = line.strip().split(':')
                if stored_username == username:
                    print("Username already exists.")
                    return False  # Username already exists
    # Hash the password
    hashed_password = hash_password(password)
    # Append new user to the file 
    with open(USER_DATA_FILE, 'a') as file:
        file.write(f"{username}:{hashed_password}\n")

    print(f"User {username} registered successfully.")
    return True

In [6]:
# Implement the User existsence check
def user_exists(username):
    """
    Checks if a username already exists in the user data file.

    Args:
        username (str): The username to check.
    Returns:
        bool: True if the user exists, False otherwise.
    """
    if os.path.exists(USER_DATA_FILE):
        with open(USER_DATA_FILE, 'r') as file:
            for line in file:
                stored_username, _ = line.strip().split(':')
                if stored_username == username:
                    return True  # User exists
    return False  # User does not exist

In [7]:
# Implementing User login
def login_user(username, password):
    """
    Authenticates a user by verifying their username password.

    Args:
        username (str): The username to authenticate.
        password (str): The plaintext password to verify.
    Returns:
        bool: True if authentication successful, False otherwise.
    """
    # Handle the case where no users are registered yet
    if not os.path.exists(USER_DATA_FILE):
        print("No users registered yet.")
        return False
    # Search the username in the file 
    with open(USER_DATA_FILE, 'r') as file:
        for line in file:
            line = line.strip()
            if not line:
                continue  # Skip empty lines
            stored_username, stored_hash = line.split(':')
            # If username matches, verify the password
            if stored_username == username:
                if verify_password(password, stored_hash):
                    print(f"User {username} logged in successfully.")
                    return True
                else:
                    print("Invalid password.")
                    return False
    # If we reach here, username was not found
    print("Username not found.")
    return False

In [8]:
# Implement input validation 
def validate_username(username):
    """
    Validates the username format.

    Args:
        username (str): The username to validate.
    Returns:
        tuple(bool, str) - (is_valid, error_message).
    """
    if not username:
        return False, "Username cannot be empty."
    # length check
    if len(username) < 3 or len(username) > 20:
        return False, "Username must be between 3 and 20 characters."
    # only letters, numbers
    if not username.isalnum():
        return False, "Username can only contain letters and numbers."
    return True, ""
def validate_password(password):
    """Vlidates password strength.
    Args:
        password (str): The password to validate.
    Returns:
        tuple(bool, str) - (is_valid, error_message).
    """
    if not password:
        return False, "Password cannot be empty."
    # length check
    if len(password) < 8 or len(password) > 50:
        return False, "Password must be between 8 and 50 characters."
    # atleast one lowercase, one uppercase, one digit
    if (not any(c.islower() for c in password)):
        return False, "Password must contain at least one lowercase letter."
    if (not any(c.isupper() for c in password)):
        return False, "Password must contain at least one uppercase letter."
    if (not any(c.isdigit() for c in password)):
        return False, "Password must contain at least one digit."
    return True, ""


In [ ]:
# Implementing the Main Menu 
def display_menu():
    """Displays the main menu options."""
    print("\n" + "="*50)
    print(" Secure Authentication System")
    print("="*50)
    print("\n[1] Register a new user")
    print("[2] Login")
    print("[3] Exit")
    print("-"*50)

def main():
    """Main program loop."""
    print("\nWelcome to the Week 7 Authentication System!")
    
    while True:
        display_menu()
        choice = input("\nPlease select an option (1-3): ").strip()
        
        if choice == '1':
            # Registration flow
            print("\n--- USER REGISTRATION ---")
            username = input("Enter a username: ").strip()
            
            # Validate username
            is_valid, error_msg = validate_username(username)
            if not is_valid:
                print(f"Error: {error_msg}")
                continue
            
            password = input("Enter a password: ").strip()
            
            # Validate password
            is_valid, error_msg = validate_password(password)
            if not is_valid:
                print(f"Error: {error_msg}")
                continue
            
            # Confirm password
            password_confirm = input("Confirm password: ").strip()
            if password != password_confirm:
                print("Error: Passwords do not match.")
                continue
            
            # Register the user
            register_user(username, password)
        
        elif choice == '2':
            # Login flow
            print("\n--- USER LOGIN ---")
            username = input("Enter your username: ").strip()
            password = input("Enter your password: ").strip()
            
            #Attempt login
            if login_user(username, password):
                print("\nYou are now logged in.")
                # Optional: Ask if they want to logout or exit
                input(f"\nSuccess: welcome[{username}]!")
        elif choice == '3':
            print("\nThank you for using the authentication system.")
            print("Exiting...")
            break
        else:
            print("\nError: Invalid option. Please select 1, 2, or 3.")
if __name__ == "__main__":
    main()
        


Welcome to the Week 7 Authentication System!

 Secure Authentication System

[1] Register a new user
[2] Login
[3] Exit
--------------------------------------------------

--- USER REGISTRATION ---
Error: Password must contain at least one uppercase letter.

 Secure Authentication System

[1] Register a new user
[2] Login
[3] Exit
--------------------------------------------------

--- USER REGISTRATION ---
Error: Password must contain at least one digit.

 Secure Authentication System

[1] Register a new user
[2] Login
[3] Exit
--------------------------------------------------

--- USER REGISTRATION ---
Error: Password must contain at least one digit.

 Secure Authentication System

[1] Register a new user
[2] Login
[3] Exit
--------------------------------------------------

--- USER LOGIN ---
Username not found.

 Secure Authentication System

[1] Register a new user
[2] Login
[3] Exit
--------------------------------------------------

Thank you for using the authentication syste